In [ ]:
import json
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
import time
from IPython.display import clear_output

In [ ]:
run_dir = "/path/to/carla/run"

# Load main vehicle log
with open(os.path.join(run_dir, "vehicle_log.json")) as f:
    frames = json.load(f)

print(f"Total frames: {len(frames)}")

# Pretty print the first frame
pretty_frame_1 = json.dumps(frames[0], indent=4)
print("First frame example:\n", pretty_frame_1)

# Pretty print the second frame
pretty_frame_2 = json.dumps(frames[1], indent=4)
print("Second frame example:\n", pretty_frame_2)

# Pretty print the last frame
pretty_frame_3 = json.dumps(frames[-1], indent=4)
print("Last frame example:\n", pretty_frame_3)

In [ ]:
with open(os.path.join(run_dir, "config.json")) as f:
    config = json.load(f)

pretty_config = json.dumps(config, indent=4)
print("Config:\n", pretty_config)

In [ ]:
image_dir = os.path.join(run_dir, "images", "front")
images = sorted(glob.glob(os.path.join(image_dir, "*.png")))

print(f"Total images: {len(images)}")
print("First 5 images:", images[:5])

# Check that image filenames in log match actual files
image_names_from_log = [frame["image_filename"] for frame in frames]
missing = [name for name in image_names_from_log if not os.path.exists(os.path.join(image_dir, name))]
print("Missing images (should be empty):", missing)

In [ ]:
timestamps = [f["timestamp"] for f in frames]
speeds = [f["speed_kmh"] for f in frames]
steers = [f["control"]["steer"] for f in frames]
brakes = [f["control"]["brake"] for f in frames]

plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.plot(timestamps, speeds)
plt.title("Speed (km/h) vs Time")
plt.xlabel("Time (s)"); plt.ylabel("Speed (km/h)")

plt.subplot(1, 3, 2)
plt.plot(timestamps, steers)
plt.title("Steering vs Time")
plt.xlabel("Time (s)"); plt.ylabel("Steering")

plt.subplot(1, 3, 3)
plt.plot(timestamps, brakes)
plt.title("Brake vs Time")
plt.xlabel("Time (s)"); plt.ylabel("Brake")

plt.tight_layout()
plt.show()

In [ ]:
n = 5
plt.figure(figsize=(15, 3))
for i in range(n + 1):
    img_path = os.path.join(image_dir, frames[i]["image_filename"])
    img = Image.open(img_path)
    plt.subplot(1, n, i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Frame {frames[i]['frame']}")
plt.show()

In [ ]:
with open(os.path.join(run_dir, "collisions.json")) as f:
    collisions = json.load(f)
print(f"Total collisions in run: {len(collisions)}")
if collisions:
    print("First collision:", collisions[0])

In [ ]:
xs = [f["location"]["x"] for f in frames]
ys = [f["location"]["y"] for f in frames]

plt.figure(figsize=(6, 6))
plt.plot(xs, ys, marker='o', markersize=2)
plt.title("Vehicle Route in World Coordinates")
plt.xlabel("x"); plt.ylabel("y")
plt.axis('equal')
plt.show()

In [ ]:
def play_video(image_paths, step=3, frame_delay=0.05, figsize=(15, 8)):
    """
    Animates a sequence of images in a Jupyter Notebook cell to simulate a video.

    Args:
        image_paths (list): A list of file paths for the images to be displayed.
        step (int): The interval at which to sample frames from the image list.
                      A higher number means a faster animation. Defaults to 3.
        frame_delay (float): The time in seconds to pause between each frame.
                               Defaults to 0.05.
        figsize (tuple): The (width, height) in inches for the displayed image.
                         Defaults to (15, 8).
    """
    try:
        for i in range(0, len(image_paths), step):
            clear_output(wait=True)

            img_path = image_paths[i]
            img = Image.open(img_path)

            plt.figure(figsize=figsize)
            plt.imshow(img)
            plt.title(f"Frame {i} of {len(image_paths)}")
            plt.axis('off')
            plt.show()

            time.sleep(frame_delay)
    except KeyboardInterrupt:
        print("Animation stopped by user.")

# play_video(images)

In [ ]:
def show_all_camera_views(frames, run_dir, index_to_show=100, cameras=None, figsize=(12, 8)):
    """
    Display all camera views for a given frame index.

    Args:
        frames (list): List of frame dictionaries containing 'image_filename' and 'frame' keys.
        run_dir (str): Path to the run directory containing the images.
        index_to_show (int): Index of the frame to display.
        cameras (list or None): List of camera names. Defaults to ["front", "front_left", "front_right", "rear"].
        figsize (tuple): Figure size for the plot.
    """
    if cameras is None:
        cameras = ["front", "front_left", "front_right", "rear"]

    # Get the image filename from the log for the specified index
    try:
        image_filename = frames[index_to_show]["image_filename"]
    except IndexError:
        print(f"Error: Index {index_to_show} is out of bounds. There are only {len(frames)} frames.")
        image_filename = None

    if image_filename:
        plt.figure(figsize=figsize)

        # Loop through the cameras and display each image in a subplot
        for i, camera in enumerate(cameras):
            # Assumes a directory structure like /run_001/images/front_left/, etc.
            image_path = os.path.join(run_dir, "images", camera, image_filename)

            plt.subplot(2, 2, i + 1)

            try:
                img = Image.open(image_path)
                plt.imshow(img)
                plt.title(f"{camera.replace('_', ' ').title()}\nFrame {frames[index_to_show]['frame']}")
            except FileNotFoundError:
                # If an image is missing, display a black placeholder
                plt.imshow(Image.new('RGB', (400, 300)))
                plt.title(f"{camera.replace('_', ' ').title()}\nImage Not Found")

            plt.axis('off')

        plt.suptitle(f"All Camera Views for Index: {index_to_show}", fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout for the main title
        plt.show()

# show_all_camera_views(frames, run_dir, index_to_show=100)